In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget https://downloads.apache.org/spark/spark-2.4.8/spark-2.4.8-bin-hadoop2.7.tgz
!tar -xvf spark-2.4.8-bin-hadoop2.7.tgz
!pip install findspark
!pip install pyspark
!wget "https://repo1.maven.org/maven2/org/apache/spark/spark-streaming-kafka-0-8-assembly_2.11/2.4.8/spark-streaming-kafka-0-8-assembly_2.11-2.4.8.jar"
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.8-bin-hadoop2.7"
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /content/spark-streaming-kafka-0-8-assembly_2.11-2.4.8.jar pyspark-shell'

In [18]:
import findspark
findspark.init()

In [19]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.ml.feature import Normalizer, StandardScaler
import random
import pyspark
import sys
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
from uuid import uuid1
import time
import math
from pyspark.sql.functions import *
import pandas as pd

In [20]:
kafka_topic_name = "harry-potter-kvs"
kafka_bootstrap_servers = 'ec2-13-229-46-113.ap-southeast-1.compute.amazonaws.com:9092'

In [265]:
sc = pyspark.SparkContext()
ssc = StreamingContext(sc,20)

In [266]:
kvs = KafkaUtils.createStream(ssc, kafka_bootstrap_servers, 'spark-streaming-consumer', {kafka_topic_name:1}) 
kvs = KafkaUtils.createDirectStream(ssc, [kafka_topic_name], {"metadata.broker.list": kafka_bootstrap_servers})
kvs = KafkaUtils.createDirectStream(ssc, [kafka_topic_name], {
                        'bootstrap.servers':kafka_bootstrap_servers,
                        'group.id':'test-group',
                        'auto.offset.reset':'largest'})

In [267]:
w = kvs.window(20)

count = w.count()

map1 = w.flatMap(lambda x: [((x[0],i),1) for i in x[1].split()])

reduce = map1.reduceByKey(lambda x,y:x+y)

tf = reduce.map(lambda x: (x[0][1],(x[0][0],x[1])))

map3 = reduce.map(lambda x: (x[0][1],(x[0][0],x[1],1)))

map4 = map3.map(lambda x:(x[0],x[1][2]))

reduce2 = map4.reduceByKey(lambda x,y:x+y)

idf=reduce2.map(lambda x: (x[0],math.log10(347/x[1])))

rdd=tf.join(idf)

rdd=rdd.map(lambda x: (x[1][0][0],(x[0],x[1][0][1],x[1][1],x[1][0][1]*x[1][1])))

rdd=rdd.map(lambda x: (x[0],x[1][0],x[1][1],x[1][2],x[1][3]))

rdd=rdd.map(lambda x: (x[0],(x[1],x[4])))

rdd=rdd.reduceByKey(lambda x,y: x if x[1] > y[1] else y)

rdd2 = rdd.transform((lambda x: x.sortBy(lambda x:(int(x[0][4:])))))

# important_w = rdd2.transform(lambda rdd:sc.parallelize(rdd.take(2000)))

# important_w = important_w.transform((lambda x:x.sortBy(lambda x:(int(x[0][-1]))).filter(lambda x: x[4]>3)))

# important_w.pprint(2000)

count.pprint()
rdd2.pprint(400)

In [268]:
ssc.start()

# stream will run for 100 sec
ssc.awaitTerminationOrTimeout(100)

-------------------------------------------
Time: 2021-09-24 17:53:00
-------------------------------------------
347

-------------------------------------------
Time: 2021-09-24 17:53:00
-------------------------------------------
('page2', ('dursley', 6.549402767062999))
('page3', ('dursley', 11.788924980713396))
('page4', ('dursley', 10.479044427300797))
('page5', ('cloaks', 4.126416440142423))
('page6', ('dursley', 6.549402767062999))
('page7', ('owls', 8.185429294411154))
('page8', ('dursley', 13.098805534125997))
('page9', ('cat', 7.494683948163243))
('page10', ('street', 7.131930612420185))
('page11', ('shooting', 3.8765389669258226))
('page12', ('lemon', 7.753077933851645))
('page13', ('dumbledore', 4.697684285419283))
('page14', ('kill', 5.844592914972996))
('page15', ('famous', 3.929641660237799))
('page16', ('motorcycle', 7.753077933851645))
('page17', ('c', 3.68271894090971))
('page18', ('street', 4.279158367452111))
('page19', ('knowing', 4.91171846339679))
('page20', ('p

False

In [264]:
ssc.stop()
sc.stop()